In [2]:
"""Swiss Metro Price Changes by Age -- Base case -- other

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',-0.215775,-20,20,0,'ASC_CAR')
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1,'ASC_SBB')
ASC_SM	 = Beta('ASC_SM',0.122635,-20,20,0,'ASC_SM')
B_CAR_COST	 = Beta('B_CAR_COST',-0.007514,-10,10,0,'B_CAR_COST')
B_HE	 = Beta('B_HE',-0.005887,-10,10,0,'B_HE')
B_SM_COST	 = Beta('B_SM_COST',-0.011591,-10,10,0,'B_SM_COST')
B_CAR_TIME	 = Beta('B_CAR_TIME',-0.013063,-10,10,0,'B_CAR_TIME')
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',-0.008814,-10,10,0,'B_TRAIN_TIME')
B_SM_TIME	 = Beta('B_SM_TIME',-0.010993,-10,10,0,'B_SM_TIME')
B_TRAIN_COST	 = Beta('B_TRAIN_COST',-0.029349,-10,10,0,'B_TRAIN_COST')
B_SENIOR	 = Beta('B_SENIOR',-1.864569,-10,10,0,'B_SENIOR')
B_GA	 = Beta('B_GA',0.518405,-10,80,0,'B_GA')
B_FIRST	 = Beta('B_FIRST',0.261506,-10,80,0,'B_FIRST')

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Exclude
# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 1  ) + (  AGE == 5  ) + (  AGE == 6  ))>0
database.remove(exclude)

#No need for estimating the model (it is already estimated). We want to obtain the individual probabilities and the market shares
prob1 = models.logit(V,av,1)
prob2 = models.logit(V,av,2)
prob3 = models.logit(V,av,3)
#Instead of reporting the choice in the simulation file, the probability of the chosen can be printed

#Simulation output
simulate = {'choice': CHOICE,
            '01 Prob. Train': prob1,
            '02 Prob. SM': prob2,
            '03 Prob. Car': prob3,
            '04 Elast SM_COST Train': Derive(prob1,'SM_COST')*SM_COST/prob1,
            '05 Elast SM_COST SM': Derive(prob2,'SM_COST')*SM_COST/prob2,
            '06 Elast SM_COST Car': Derive(prob3,'SM_COST')*SM_COST/prob3}

biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "MNL_SM_price_base_other"
results = biogeme.simulate()
print("Results=",results.describe())

07-04-2019:13:41:03,343 WARNING  [biogeme.py:143] The chosen alternative is not available for the following observations (rownumber[choice]): 9[3]-10[3]-11[3]-12[3]-13[3]-14[3]-15[3]-16[3]-17[3]-36[3]-37[3]-38[3]-39[3]-40[3]-41[3]-42[3]-43[3]-44[3]-81[3]-82[3]-83[3]-84[3]-85[3]-86[3]-87[3]-88[3]-89[3]-90[3]-91[3]-92[3]-93[3]-94[3]-95[3]-96[3]-97[3]-98[3]-99[3]-100[3]-101[3]-102[3]-103[3]-104[3]-105[3]-106[3]-107[3]-108[3]-109[3]-110[3]-111[3]-112[3]-113[3]-114[3]-115[3]-116[3]-126[3]-127[3]-128[3]-129[3]-130[3]-131[3]-132[3]-133[3]-134[3]-180[3]-181[3]-182[3]-183[3]-184[3]-185[3]-186[3]-187[3]-188[3]-207[3]-208[3]-209[3]-210[3]-211[3]-212[3]-213[3]-214[3]-215[3]-234[3]-235[3]-236[3]-237[3]-238[3]-239[3]-240[3]-241[3]-242[3]-279[3]-280[3]-281[3]-282[3]-283[3]-284[3]-285[3]-286[3]-287[3]-288[3]-289[3]-290[3]-291[3]-292[3]-293[3]-294[3]-295[3]-296[3]-297[3]-298[3]-299[3]-300[3]-301[3]-302[3]-303[3]-304[3]-305[3]-306[3]-307[3]-308[3]-309[3]-310[3]-311[3]-312[3]-313[3]-314[3]-315[3]-316[3]-

Results=             choice  01 Prob. Train  02 Prob. SM  03 Prob. Car  \
count  5904.000000    5.904000e+03  5904.000000   5904.000000   
mean      2.203421    9.798152e-02     0.615112      0.286906   
std       0.576032    9.087751e-02     0.174497      0.207416   
min       1.000000    1.289617e-07     0.000838      0.000000   
25%       2.000000    2.899079e-02     0.506893      0.126844   
50%       2.000000    7.075455e-02     0.623107      0.267613   
75%       3.000000    1.353334e-01     0.741560      0.408999   
max       3.000000    4.615759e-01     0.995591      0.999098   

       04 Elast SM_COST Train  05 Elast SM_COST SM  06 Elast SM_COST Car  
count             5904.000000          5904.000000           5904.000000  
mean                 0.730520            -0.535235              0.672743  
std                  0.516718             0.637097              0.520095  
min                  0.000000            -8.891787              0.000000  
25%                  0.373647 